# Parcel Data Exploration

## Step 1 - Toy dataset
Used pandas to filter the Parcel sheet to a single street. Using Hampden Street since that's what was filtered when the parcel data was first sent. Filtered using pandas and saved as CSV (code removed because only needed once).

Interestingly, the street name was not `"HAMPDEN ST"` but rather `"HAMPDEN ST                         "` (lots of trailing whitespace. Should keep an eye out for similar issues moving forward. Also had a warning thrown by pandas because in the full dataset there are columns with non-uniform datatypes.

In [1]:
import pandas as pd
import numpy as np

In [2]:
FILEPATH = "./data_explore/Parcel_Holyoke.csv"

In [3]:
df = pd.read_csv(FILEPATH)

/Users/conorlyman/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.head()

,Unnamed: 0,PCL ID,Property Account Number,ST NU,ST ALT,LOCATION,CONDO_UNIT,COMPLEX_NAME,OWNER,OWNER 2,...,Number of Units,USE_CLASS,Vintage,Heat_Fuel,Heat_System,FLA.1,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102
0,1,001-01-001,,,,ST KOLBE DR,,,HOLYOKE CITY OF ...,GAS + ELECTRIC DEPT ...,...,0,Commercial,POST2000,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2,001-01-002,0,,REAR,ST KOLBE DR,,,MASSACHUSETTS DOT ...,,...,0,Commercial,POST2000,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,3,001-01-003,0,,REAR,ST KOLBE DR,,,HOLYOKE CITY OF ...,PARKS + RECREATION ...,...,1,Commercial,1950-1979,NONE,NONE,240.0,NaN,NaN,NaN,NaN
3,4,001-01-004,0,,REAR,ST KOLBE DR,,,HOLYOKE CITY OF ...,PARKS + RECREATION ...,...,0,Commercial,POST2000,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,5,001-01-005,0,,REAR,ST KOLBE DR,,,HOLYOKE CITY OF ...,PARKS + RECREATION ...,...,0,Commercial,POST2000,NaN,NaN,0.0,NaN,NaN,NaN,NaN


### Note
Note that "Unnamed: 0" is the original row number (so we can probably ignore) and "Unnamed: 0.1" is (I'm guessing) the original entry number (can probably also ignore).

Also note that many of these cols were added by Mike and can probably be removed as the considered input data.

## Step 2 - Explore columns

Let's explore the columns here and define key parameters and metadata. 

In [5]:
[i for i in df.columns]

['Unnamed: 0',
 'PCL ID',
 'Property Account Number',
 'ST NU',
 'ST ALT',
 'LOCATION',
 'CONDO_UNIT',
 'COMPLEX_NAME',
 'OWNER',
 'OWNER 2',
 'OWNER 3',
 'MAILING',
 'NULL',
 'CITY',
 'ST',
 'ZIP',
 'NULL.1',
 'ZONING',
 'NULL.2',
 'CENSUS',
 'UTILITY',
 'NULL.3',
 'NULL.4',
 'TRAFFIC',
 'LUC',
 'YEAR BLT',
 '?PERMIT? ',
 'ACRES',
 'SKETCH AREA',
 'DESC',
 'NULL.5',
 'NULL.6',
 'FLA',
 'Number of Buildings (Property Cards) ',
 'ST HT',
 'FULL BA',
 'ROOMS',
 'BRS',
 'BATHS',
 'HALF BATHS',
 'Unnamed: 40',
 'COND',
 'KITCHENS',
 'Building Conditions',
 'Fireplaces',
 'Unnamed: 45',
 'Solar Hot Water',
 'Central Vacuum',
 'HEAT TYPE',
 'HEAT FUEL',
 'PCT_AIR_CONDITIONED',
 'BSMT AREA',
 'BSMT_ALT_FINISH',
 'ROOF',
 'ROOF_COVER',
 'SIDING',
 'WALL',
 'Unnamed: 57',
 'Unnamed: 58',
 'Unnamed: 59',
 'Unnamed: 60',
 'EXT',
 'FLOORS',
 '?CURRENT ASSESSMENT?',
 'GRADE',
 'COND.1',
 'BK / PG',
 'DATE',
 'SALES PRICE',
 'ARMS Sales',
 'GRANTOR LAST NAME',
 'BK / PG (2)',
 'DATE (2)',
 'Sales Pr

In [6]:
# Data Description
data_description = {
    "PCL ID": "The parcel ID string",
    "PCL_ID_Tr": "The parcel ID string trimmed for leading/trialing whitespace",
    "HEAT TYPE": "The type of heating system of the parcel",
    "HEAT_TYPE_Tr": "HEAT TYPE string trimmed for leading/trailing whitepace",
    "HEAT FUEL": "The type of heating fuel",
    "HEAT_FUEL_Tr": "HEAT FUEL string trimmed",
    "Heat_Fuel": "A copy of HEAT_FUEL_Tr",
    "DESC": "A description of the parcel",
    "DECS.1": "A copy of DESC",
    "Building Type": "A simplification of DESC, using a customized mapping",
    "Typology": "A copy of Building Type",
    "Decade Built": "Maps YEAR BLT to a decade",
    "YEAR BLT": "The year built",
    "FLA": "The floor are in square feet (?)",
    "FLA.1": "A copy of FLA",
    "Heat_System": "A mapping of HEAT_TYPE_Tr, based on a customized mapping",
    "Vintage": "A copy of Age Category",
    "Age Category": "Custom mapping of YEAR BLT",
    "USE_CLASS": "Custom mapping of DESC and LUC",
    "Number of Units": "Number of units, based on Typology and KITCHENS as a proxy for units",
    "KITCHENS": "Number of kitchens",
    "Numb": "Empty",
    "StateTrimmed": "Empty",
    "STATE CODE": "Empty",
    "ADJ AREA": "Assuming this is adjusted area of ? in units of ?",
    "TOTAL VAL": "Total property value? Appears to be a sum of BUILD VAL, YARD ITEMS, and LAND VAL",
    "BUILD VAL": "Total building value (?)",
    "YARD ITEMS": "What? Some kind of monetary value",
    "LAND VAL": "Total land value (?)",
    "GRANTOR LAST NAME": "Not sure what a grantor is (previous owner?), but probably not important",
    "GRANTOR LAST NAME (2)": "Same comment as GRANTOR LAST NAME",
    "ARMS Sales": "Not sure; probably not important",
    "ARMS Sales (2)": "Same comment as ARMS Sales",
    "SALES PRICE": "Probably the selling price of the property",
    "Sales Price (2)": "Not sure; probably not important",
    "DATE": "Sales date?",
    "DATE (2)": "Sales date?",
    "BK / PG": "No idea",
    "BK / PG (2)": "No idea",
    "COND": "Condition",
    "COND.1": "Condition, again",
    "GRADE": "Seems also like a condition; it's a letter grade",
    "?CURRENT ASSESSMENT?": "Seems like assessment year",
    "FLOORS": "Floor material type",
    "EXT": "Exterior building material?",
    "WALL": "Wall material",
    "SIDING": "Siding material",
    "ROOF": "Roof type?",
    "ROOF_COVER": "Roof cover material",
    "BSMT AREA": "The basement area, in square feet?",
    "BSMT_ALT_FINISH": "From Mike, appears to be finished basement are, in square feet?",
    "PCT_AIR_CONDITIONED": "Percent of building that is air conditioned? Does it include window units?",
    "Central Vacuum": "Boolean var for presence of a central vacuum in the building?",
    "Solar Hot Water": "Boolean var for presence of a solar water heater",
    "Fireplaces": "Number of fireplaces",
    "Building Conditions": "The condition of the building",
    "FULL BA": "Number of full baths",
    "ROOMS": "Number of rooms",
    "BRS": "Number of bedrooms",
    "BATHS": "Number of all bathrooms?",
    "HALF BATHS": "Number of half baths",
    "ST HT": "Not sure",
    "Number of Buildings (Property Cards) ": "Number of buildings on the parcel (with fun trailing whitespace)",
    "SKETCH AREA": "No idea",
    "ACRES": "Acres of the parcel",
    "?PERMIT? ": "Not sure since it's mostly a blank col",
    "LUC": "Some sort of ID number",
    "TRAFFIC": "Car traffic by the parcel?",
    "UTILITY": "The utility for the parcel? Only entry is NONE or PUBLIC",
    "CENSUS": "The census tract?",
    "ZONING": "The zoning of the lot",
    "OWNER": "Owner last name",
    "OWNER 2": "Owner last name",
    "OWNER 3": "Owner last name",
    "CONDO_UNIT": "Whether or not there's a condo unit?",
    "COMPLEX_NAME": "The complex name?",
    "ST NU": "The street number",
    "ST ALT": "Alternative street number",
    "LOCATION": "Street name",
    "MAILING": "Mailing address (just street number and name)",
    "CITY": "City",
    "ST": "State",
    "ZIP": "Zip code",
    "Property Account Number": "Property account number",
    "NULL and Unnamed": "Lots of null or unnamed cols",
}

## Step 3 - Clean dataset
We're going to clean the dataset down to just the "raw" input data that we need (removing null cols and calculated values). Afterwards, we'll write functions to calculate the other cols and save an updated dataset.

In [7]:
def create_raw_dataset(raw_data_cols: list, input_filepath: str, output_filepath: str) -> None:
    df = pd.read_csv(input_filepath)
    df = df.reindex(columns=raw_data_cols)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df.to_csv(output_filepath, index=False)

In [8]:
cols = [
    "PCL ID",
    "HEAT TYPE",
    "HEAT FUEL",
    "DESC",
    "YEAR BLT",
    "FLA",
    "KITCHENS",
    "ADJ AREA",
    "TOTAL VAL",
    "BUILD VAL",
    "YARD ITEMS",
    "LAND VAL",
    "GRANTOR LAST NAME",
    "GRANTOR LAST NAME (2)",
    "ARMS Sales",
    "ARMS Sales (2)",
    "SALES PRICE",
    "Sales Price (2)",
    "DATE",
    "DATE (2)",
    "BK / PG",
    "BK / PG (2)",
    "COND",
    "COND.1",
    "GRADE",
    "?CURRENT ASSESSMENT?",
    "FLOORS",
    "EXT",
    "WALL",
    "SIDING",
    "ROOF",
    "ROOF_COVER",
    "BSMT AREA",
    "BSMT_ALT_FINISH",
    "PCT_AIR_CONDITIONED",
    "Central Vacuum",
    "Solar Hot Water",
    "Fireplaces",
    "Building Conditions",
    "FULL BA",
    "ROOMS",
    "BRS",
    "BATHS",
    "HALF BATHS",
    "ST HT",
    "Number of Buildings (Property Cards) ",
    "SKETCH AREA",
    "ACRES",
    "?PERMIT? ",
    "LUC",
    "TRAFFIC",
    "UTILITY",
    "CENSUS",
    "ZONING",
    "OWNER",
    "OWNER 2",
    "OWNER 3",
    "CONDO_UNIT",
    "COMPLEX_NAME",
    "ST NU",
    "ST ALT",
    "LOCATION",
    "MAILING",
    "CITY",
    "ST",
    "ZIP",
    "Property Account Number",
]

In [9]:
input_filepath = "./data_explore/Parcel_Holyoke.csv"
output_filepath = "./database/raw_parcel_data.csv"

In [10]:
create_raw_dataset(cols, input_filepath, output_filepath)

/var/folders/q4/7911fbv57wv3c68srp1cdlkm0000gn/T/ipykernel_15640/507058006.py:1: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  create_raw_dataset(cols, input_filepath, output_filepath)


## Step 4 - Calculated values
Now, we will calculate values of interest and save them to our database using repeatable functions.

Functions we need to add are:
* Building Type: A simplification of DESC, using a customized mapping
* Decade Built: Maps YEAR BLT to a decade
* Heat_System: A mapping of HEAT_TYPE, based on a customized mapping
* Age Category: Custom mapping of YEAR BLT
* USE_CLASS: Custom mapping of DESC and LUC
* Number of Units: Number of units, based on Typology and KITCHENS as a proxy for units
    * Going to use FLA instead of Typology (if FLA is 0, then you can assume no building or units)

### Number of units

In [11]:
df = pd.read_csv(output_filepath)
df.head()

,PCL ID,HEAT TYPE,HEAT FUEL,DESC,YEAR BLT,FLA,KITCHENS,ADJ AREA,TOTAL VAL,BUILD VAL,...,CONDO_UNIT,COMPLEX_NAME,ST NU,ST ALT,LOCATION,MAILING,CITY,ST,ZIP,Property Account Number
0,001-01-001,NaN,NaN,NaN,NaN,0.0,0,NaN,8300.0,0.0,...,NaN,NaN,NaN,NaN,ST KOLBE DR,99 SUFFOLK ST,HOLYOKE,MA,01040,NaN
1,001-01-002,NaN,NaN,NaN,NaN,0.0,0,NaN,40500.0,0.0,...,NaN,NaN,NaN,REAR,ST KOLBE DR,10 PARK PLAZA,BOSTON,MA,02116,0.0
2,001-01-003,NONE,NONE,PMP/VLV HS,1970.0,240.0,0,240.0,62400.0,13800.0,...,NaN,NaN,NaN,REAR,ST KOLBE DR,536 DWIGHT ST,HOLYOKE,MA,01040,0.0
3,001-01-004,NaN,NaN,NaN,NaN,0.0,0,NaN,33700.0,0.0,...,NaN,NaN,NaN,REAR,ST KOLBE DR,536 DWIGHT ST,HOLYOKE,MA,01040,0.0
4,001-01-005,NaN,NaN,NaN,NaN,0.0,0,NaN,37100.0,0.0,...,NaN,NaN,NaN,REAR,ST KOLBE DR,536 DWIGHT ST,HOLYOKE,MA,01040,0.0


In [12]:
def get_number_of_units(_x):
    floor_area = _x["FLA"]
    num_kitchens = _x["KITCHENS"]
    
    if num_kitchens > 0 and floor_area > 0:
        return num_kitchens
    return 0

In [13]:
df["Number of units"] = df.apply(get_number_of_units, axis=1)

### Heat system

In [14]:
def get_heat_system(_x):
    mapping = {
        None: None,
        "STEAM": "STEAM",
        "FORCED H/A": "DUCTED",
        "FORCED H/W": "NON-DUCTED",
        "UNIT HTRS": "NON-DUCTED",
        "ELECTRC BB": "NON-DUCTED",
        "RADIANT HW": "NON-DUCTED",
        "GRAVTY H/A": "NON-DUCTED",
        "FLOOR FURN": "NON-DUCTED",
        "NOT DUCTED": "NON-DUCTED",
        "AVERAGE": "NON-DUCTED",
        "WALL UNIT": "NON-DUCTED",
    }
    
    return mapping.get(_x["HEAT TYPE"])

In [15]:
df["Heat system"] = df.apply(get_heat_system, axis=1)

### Building type

In [16]:
building_type_mapping_file = "./database/mappings/building_type_mapping.csv"
df_build_type_mapping = pd.read_csv(building_type_mapping_file)

In [17]:
df_build_type_mapping

,DESC,Building Type
0,PMP/VLV HS,Office
1,2 FAMILY,Small Multifamily
2,4-8 FAM,Large Residential
3,3 FAMILY,Small Multifamily
4,APRTMNT-GN,Large Residential
...,...,...
79,SUPER MKT,Supermarket
80,NIGHT CLUB,Resturant
81,BOWLING AL,Assembly
82,RINK,Assembly


In [18]:
mapping_dict = df_build_type_mapping.set_index("DESC").to_dict()["Building Type"]

In [19]:
def set_building_type(_x, mapping_dict):
    return mapping_dict.get(_x["DESC"])

In [20]:
df["Building Type"] = df.apply(set_building_type, axis=1, args=(mapping_dict,))

### Decade built

In [21]:
df["Decade Built"] = ((df["YEAR BLT"].fillna(0) // 10) * 10).astype(int).replace(0, np.nan)

In [22]:
df

,PCL ID,HEAT TYPE,HEAT FUEL,DESC,YEAR BLT,FLA,KITCHENS,ADJ AREA,TOTAL VAL,BUILD VAL,...,LOCATION,MAILING,CITY,ST,ZIP,Property Account Number,Number of units,Heat system,Building Type,Decade Built
0,001-01-001,NaN,NaN,NaN,NaN,0.0,0,NaN,8300.0,0.0,...,ST KOLBE DR,99 SUFFOLK ST,HOLYOKE,MA,01040,NaN,0,None,None,NaN
1,001-01-002,NaN,NaN,NaN,NaN,0.0,0,NaN,40500.0,0.0,...,ST KOLBE DR,10 PARK PLAZA,BOSTON,MA,02116,0.0,0,None,None,NaN
2,001-01-003,NONE,NONE,PMP/VLV HS,1970.0,240.0,0,240.0,62400.0,13800.0,...,ST KOLBE DR,536 DWIGHT ST,HOLYOKE,MA,01040,0.0,0,None,Office,1970.0
3,001-01-004,NaN,NaN,NaN,NaN,0.0,0,NaN,33700.0,0.0,...,ST KOLBE DR,536 DWIGHT ST,HOLYOKE,MA,01040,0.0,0,None,None,NaN
4,001-01-005,NaN,NaN,NaN,NaN,0.0,0,NaN,37100.0,0.0,...,ST KOLBE DR,536 DWIGHT ST,HOLYOKE,MA,01040,0.0,0,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10518,041-01-018,NaN,NaN,NaN,NaN,0.0,0,NaN,24700.0,0.0,...,CANAL ST,11440 WEST BERNARDO COURT,SAN DIEGO,CA,92127,NaN,0,None,None,NaN
10519,037-01-011,NaN,NaN,NaN,NaN,0.0,0,NaN,15200.0,0.0,...,CANAL ST,11440 WEST BERNARDO COURT,SAN DIEGO,CA,92127,NaN,0,None,None,NaN
10520,041-01-019,NaN,NaN,NaN,NaN,0.0,0,NaN,21000.0,0.0,...,NORTH BRIDGE ST,11440 WEST BERNARDO COURT,SAN DIEGO,CA,92127,NaN,0,None,None,NaN
10521,078-00-056,NaN,NaN,NaN,NaN,0.0,0,NaN,36300.0,0.0,...,CARTER ST,18 CARTER ST,HOLYOKE,MA,01040,NaN,0,None,None,NaN


### Use class

In [23]:
'''
Mapped using LUC and DESC.
Note that there are some entries for DESC that are just a bunch of whitespace
    in the original sheet, but here are NaN.
'''

'\nMapped using LUC and DESC.\nNote that there are some entries for DESC that are just a bunch of whitespace\n    in the original sheet, but here are NaN.\n'

In [24]:
map_df = pd.read_csv("./database/mappings/map_LUCandDESC_UseClass.csv")

In [25]:
map_df

,LUC,DESC,USE_CLASS,mapping_label
0,13,MX USE APT,Commercial,13 - MX USE APT
1,13,COMM BLOCK,Commercial,13 - COMM BLOCK
2,13,MX USE < 9,Commercial,13 - MX USE < 9
3,13,VICTORIAN,Single Family,13 - VICTORIAN
4,13,2 FAMILY,Small Multifamily,13 - 2 FAMILY
...,...,...,...,...
404,972,PMP/VLV HS,Commercial,972 - PMP/VLV HS
405,972,TRUCK TERM,Commercial,972 - TRUCK TERM
406,972,OFFICE,Commercial,972 - OFFICE
407,980,NaN,Commercial,980 -


In [26]:
mapping_dict = map_df[["USE_CLASS", "mapping_label"]].set_index("mapping_label").to_dict()["USE_CLASS"]

In [27]:
def map_use_class(_x, mapping):
    label = str(_x["LUC"]) + ' - ' + str(_x["DESC"])
    return mapping.get(label)

In [28]:
# Hack-y way to handle NaNs in DESC is to just fill all NaN this once
df["Use Class"] = df.fillna('').apply(map_use_class, axis=1, args=(mapping_dict,))

In [29]:
df

,PCL ID,HEAT TYPE,HEAT FUEL,DESC,YEAR BLT,FLA,KITCHENS,ADJ AREA,TOTAL VAL,BUILD VAL,...,MAILING,CITY,ST,ZIP,Property Account Number,Number of units,Heat system,Building Type,Decade Built,Use Class
0,001-01-001,NaN,NaN,NaN,NaN,0.0,0,NaN,8300.0,0.0,...,99 SUFFOLK ST,HOLYOKE,MA,01040,NaN,0,None,None,NaN,Commercial
1,001-01-002,NaN,NaN,NaN,NaN,0.0,0,NaN,40500.0,0.0,...,10 PARK PLAZA,BOSTON,MA,02116,0.0,0,None,None,NaN,Commercial
2,001-01-003,NONE,NONE,PMP/VLV HS,1970.0,240.0,0,240.0,62400.0,13800.0,...,536 DWIGHT ST,HOLYOKE,MA,01040,0.0,0,None,Office,1970.0,Commercial
3,001-01-004,NaN,NaN,NaN,NaN,0.0,0,NaN,33700.0,0.0,...,536 DWIGHT ST,HOLYOKE,MA,01040,0.0,0,None,None,NaN,Commercial
4,001-01-005,NaN,NaN,NaN,NaN,0.0,0,NaN,37100.0,0.0,...,536 DWIGHT ST,HOLYOKE,MA,01040,0.0,0,None,None,NaN,Commercial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10518,041-01-018,NaN,NaN,NaN,NaN,0.0,0,NaN,24700.0,0.0,...,11440 WEST BERNARDO COURT,SAN DIEGO,CA,92127,NaN,0,None,None,NaN,Commercial
10519,037-01-011,NaN,NaN,NaN,NaN,0.0,0,NaN,15200.0,0.0,...,11440 WEST BERNARDO COURT,SAN DIEGO,CA,92127,NaN,0,None,None,NaN,Commercial
10520,041-01-019,NaN,NaN,NaN,NaN,0.0,0,NaN,21000.0,0.0,...,11440 WEST BERNARDO COURT,SAN DIEGO,CA,92127,NaN,0,None,None,NaN,Commercial
10521,078-00-056,NaN,NaN,NaN,NaN,0.0,0,NaN,36300.0,0.0,...,18 CARTER ST,HOLYOKE,MA,01040,NaN,0,None,None,NaN,Large Residential


### Age category

Age categories are:
* `"PRE1950"`
* `"1950 - 1979"`
* `"1980 - 1999"`
* `"POST2000"`

*Note that NaN values are considered post-2000*

In [30]:
pd.isnull(df.loc[0, "YEAR BLT"])

True

In [31]:
def age_mapping(_x):
    year = _x["YEAR BLT"]
    
    if pd.isnull(year):
        return "POST2000"
    
    else:
        if year < 1950:
            return "PRE1950"
        elif year >= 1950 and year < 1980:
            return "1950 - 1979"
        elif year >= 1980 and year < 2000:
            return "1980 - 1999"
        else:
            return "POST2000"
        
    return "POST2000"

In [32]:
df["Age category"] = df.apply(age_mapping, axis=1)

## Step 4 - Export

Now, export the final database, including calculated cols

In [33]:
df.to_csv("./database/final_parcel_data.csv", index=False)